# Python Code to webscrapp the charts and stats from MMS for the exam officer summary table

Requeriments for the code to run:

1. You need the list of modules in a format like: "GG4258" no spaces
2. The links for the requiered charts (scatterplot with grade distribution and scatterplot with grade distribution from previous years)  are https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/GG3214/Final+grade/GraphPage https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/GG3281/Final+grade/SubmitResults and the link for the grade table is https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/GG1002/Final+grade/ you will notice both links have the module code as parameter
3. You can adapt the code to update the list of modules per semestre but also the AY, as for now the link is consistent across modules. 
4. The main script is `ModuleGradesChartsExtractor.py` there other scrips that describe parts of the process, but I kept them just for testing asnd adapting in the future.

Now the code in here just allow you to install the requeriments in an independent python environnment, once that is done, you can just open a terminal and run: `python ModuleGradesChartsExtractor.py` or `python  module_charts_downloader.py`


In [ ]:
# pip install requests beautifulsoup4 plotly kaleido selenium webdriver-manager openpyxl pillow

In the following lines I will try to describe step by step what you need to get the information related to gardes, grades distributions charts, and estadistics for all the modules to report. This excel file is a helpful resource for the externals and module convenours to have a discussion about the performance of their modules in the corresponding semestre.

Let's begin:

1. Here I try to web scrapp the grade table from MMS for one module, just initual manual authentication (allowing the user to authenticate and avoid making this single-on authentication part of the process., therefore is easier.

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    print("=== Manual Authentication ===")
    print("1. A browser will open.")
    print("2. Login to MMS and navigate to the test module page.")
    print("3. Once you see the full table, come back and press Enter here.")
    driver.get(test_url)
    input("Press Enter when you're on the module page and see the grades table...")


def extract_table_html(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table))[0]
    return df


def main():
    module_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/GG1002/Final+grade/"
    driver = setup_driver()

    try:
        manual_login(driver, module_url)
        table_html = extract_table_html(driver)
        df = parse_html_table_to_dataframe(table_html)

        print("\n=== DataFrame Preview ===")
        print(df.head())
        print("\n=== Column Names ===")
        print(df.columns.tolist())

    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()


if __name__ == "__main__":
    main()

2. As it worked, now I have adapted the script to make a excel file and only worked with the columns I need, `Matric Number and Calculated Grade.

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    print("=== Manual Authentication ===")
    driver.get(test_url)
    input("➡️ Once you're logged in and see the grades table, press Enter here...")


def extract_table_html(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table), header=[0, 1])[0]  # Read MultiIndex headers
    return df


def filter_and_save(df, module_code):
    # Print out the available columns as tuples
    print("📋 Available columns:\n", df.columns.tolist())

    # Select only the required columns using MultiIndex tuples
    try:
        filtered_df = df[[('Student ↓↑', 'Matric Number ↓↑'), ('Result ↓↑', 'Calc Grade ↓↑')]]
    except KeyError as e:
        print("❌ Column not found. Error:", e)
        return

    # Rename columns for clarity
    filtered_df.columns = ['Matric Number', 'Calc Grade']

    # Save to Excel
    output_filename = f"{module_code}_grades.xlsx"
    filtered_df.to_excel(output_filename, sheet_name=module_code, index=False)
    print(f"✅ Grades saved to '{output_filename}'.")


def main():
    module_code = "GG1002"
    module_url = f"https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/{module_code}/Final+grade/"
    driver = setup_driver()

    try:
        manual_login(driver, module_url)
        table_html = extract_table_html(driver)
        df = parse_html_table_to_dataframe(table_html)

        filter_and_save(df, module_code)

    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()


if __name__ == "__main__":
    main()

Now, it is a matter of make it for the entire list of modules to report.

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    print("=== Manual Authentication ===")
    driver.get(test_url)
    input("➡️ Once you're logged in and see the grades table, press Enter here...")


def extract_table_html(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table), header=[0, 1])[0]  # Read MultiIndex headers
    return df


def filter_grades_dataframe(df, module_code):
    # Print out the available columns as tuples
    print(f"📋 Columns for {module_code}:\n", df.columns.tolist())

    try:
        filtered_df = df[[('Student ↓↑', 'Matric Number ↓↑'), ('Result ↓↑', 'Calc Grade ↓↑')]]
    except KeyError as e:
        print(f"❌ Columns not found in {module_code}. Error: {e}")
        return None

    filtered_df.columns = ['Matric Number', 'Calc Grade']
    return filtered_df


def main():
    module_codes = ['GG4258', 'GG3281', 'GG1002', 'GG2014', 'GG4248', 'GG4247', 'SS5103',
    'GG4254', 'GG4257', 'GG3205', 'GG3213', 'GG3214', 'GG5005', 'GG4399',
    'SD4126', 'SD4129', 'SD4133', 'SD1004', 'SD4225', 'SD2006', 'SD2100',
    'SD4110', 'SD3102', 'SD3101', 'SD4120', 'SD4125', 'SD4297', 'SD5801',
    'SD5802', 'SD5805', 'SD5806', 'SD5807', 'SD5810', 'SD5820', 'SD5821',
    'SD5811', 'SD5813', 'SD5812']
    base_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2"
    driver = setup_driver()
    all_grades = {}

    try:
        # Login once manually using the first module's URL
        login_url = f"{base_url}/{module_codes[0]}/Final+grade/"
        manual_login(driver, login_url)

        for module_code in module_codes:
            print(f"\n🔍 Processing module {module_code}...")
            module_url = f"{base_url}/{module_code}/Final+grade/"
            driver.get(module_url)

            try:
                table_html = extract_table_html(driver)
                df = parse_html_table_to_dataframe(table_html)
                filtered_df = filter_grades_dataframe(df, module_code)

                if filtered_df is not None:
                    all_grades[module_code] = filtered_df
                    print(f"✅ Data collected for {module_code}.")

            except Exception as e:
                print(f"⚠️ Error processing {module_code}: {e}")

        # Save all to one Excel workbook
        if all_grades:
            output_filename = "All_Modules_Grades.xlsx"
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                for module_code, df in all_grades.items():
                    df.to_excel(writer, sheet_name=module_code, index=False)
            print(f"\n📁 All grades saved to '{output_filename}'.")

    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()


if __name__ == "__main__":
    main()

Now is ok, but we also get basic stadistics from the web scrapping so I take them and include a new sheet with a summary data for all modules

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    print("=== Manual Authentication ===")
    driver.get(test_url)
    input("➡️ Once you're logged in and see the grades table, press Enter here...")


def extract_table_html(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table), header=[0, 1])[0]  # Read MultiIndex headers
    return df


def filter_grades_dataframe(df, module_code):
    #print(f"📋 Columns for {module_code}:\n", df.columns.tolist())

    try:
        filtered_df = df[[('Student ↓↑', 'Matric Number ↓↑'), ('Result ↓↑', 'Calc Grade ↓↑')]]
    except KeyError as e:
        print(f"❌ Columns not found in {module_code}. Error: {e}")
        return None, None

    filtered_df.columns = ['Matric Number', 'Calc Grade']

    # Split data from summary rows
    student_data = filtered_df.iloc[:-6].copy()
    summary_data = filtered_df.iloc[-6:].copy()

    # Convert summary rows into a single-row DataFrame with named index
    summary_row = summary_data.set_index('Matric Number').T
    summary_row.columns.name = None
    summary_row['Module'] = module_code
    summary_row = summary_row.set_index('Module')

    return student_data, summary_row


def main():
    module_codes = ['GG4258', 'GG3281', 'GG1002', 'GG2014', 'GG4248', 'GG4247', 'SS5103',
    'GG4254', 'GG4257', 'GG3205', 'GG3213', 'GG3214', 'GG5005', 'GG4399',
    'SD4126', 'SD4129', 'SD4133', 'SD1004', 'SD4225', 'SD2006', 'SD2100',
    'SD4110', 'SD3102', 'SD3101', 'SD4120', 'SD4125', 'SD4297', 'SD5801',
    'SD5802', 'SD5805', 'SD5806', 'SD5807', 'SD5810', 'SD5820', 'SD5821',
    'SD5811', 'SD5813', 'SD5812']
    base_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2"
    driver = setup_driver()
    all_grades = {}
    all_summaries = []

    try:
        login_url = f"{base_url}/{module_codes[0]}/Final+grade/"
        manual_login(driver, login_url)

        for module_code in module_codes:
            print(f"\n🔍 Processing module {module_code}...")
            module_url = f"{base_url}/{module_code}/Final+grade/"
            driver.get(module_url)

            try:
                table_html = extract_table_html(driver)
                df = parse_html_table_to_dataframe(table_html)
                student_data, summary_row = filter_grades_dataframe(df, module_code)

                if student_data is not None:
                    all_grades[module_code] = student_data
                    all_summaries.append(summary_row)
                    print(f"✅ Data collected for {module_code}.")

            except Exception as e:
                print(f"⚠️ Error processing {module_code}: {e}")

        # Save everything to a single Excel file
        if all_grades:
            output_filename = "All_Modules_Grades_with_Summary_050625.xlsx"
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                for module_code, df in all_grades.items():
                    df.to_excel(writer, sheet_name=module_code, index=False)

                if all_summaries:
                    summary_df = pd.concat(all_summaries)
                    summary_df.to_excel(writer, sheet_name="Summary")
            print(f"\n📁 All grades and summary saved to '{output_filename}'.")

    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()


if __name__ == "__main__":
    main()

Now I need to add the columns to compute if there is any criteria for decanal intervention, as they may vary over semestre I decided to included in excel with the formula: =IF(OR(I3<5%, I3>30%, F3<16.9, J3>=80%, D3<12, E3<12), "Needs Comment", "OK")

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    print("=== Manual Authentication ===")
    driver.get(test_url)
    input("➡️ Once you're logged in and see the grades table, press Enter here...")


def extract_table_html(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table), header=[0, 1])[0]  # Read MultiIndex headers
    return df


def filter_grades_dataframe(df, module_code):
    print(f"📋 Columns for {module_code}:\n", df.columns.tolist())

    try:
        filtered_df = df[[('Student ↓↑', 'Matric Number ↓↑'), ('Result ↓↑', 'Calc Grade ↓↑')]]
    except KeyError as e:
        print(f"❌ Columns not found in {module_code}. Error: {e}")
        return None, None

    filtered_df.columns = ['Matric Number', 'Calc Grade']

    # Split data from summary rows
    student_data = filtered_df.iloc[:-6].copy()
    summary_data = filtered_df.iloc[-6:].copy()

    # Convert grades to numeric (ignore non-numeric or missing values)
    student_data['Calc Grade'] = pd.to_numeric(student_data['Calc Grade'], errors='coerce')

    # Calculate extra statistics
    total_students = student_data['Calc Grade'].count()
    pct_gte_16_5 = (student_data['Calc Grade'] >= 16.5).sum() / total_students * 100 if total_students > 0 else 0
    pct_14_16 = ((student_data['Calc Grade'] >= 14) & (student_data['Calc Grade'] < 16)).sum() / total_students * 100 if total_students > 0 else 0

    # Format summary row from table
    summary_row = summary_data.set_index('Matric Number').T
    summary_row.columns.name = None
    summary_row['Module'] = module_code

    # Add new calculated percentages
    summary_row['% ≥ 16.5'] = round(pct_gte_16_5, 2)
    summary_row['% between 14–16'] = round(pct_14_16, 2)

    summary_row = summary_row.set_index('Module')

    return student_data, summary_row


def main():
    
    module_codes = ['GG4258', 'GG3281', 'GG1002', 'GG2014', 'GG4248', 'GG4247', 'SS5103',
    'GG4254', 'GG4257', 'GG3205', 'GG3213', 'GG3214', 'GG5005', 'GG4399',
    'SD4126', 'SD4129', 'SD4133', 'SD1004', 'SD4225', 'SD2006', 'SD2100',
    'SD4110', 'SD3102', 'SD3101', 'SD4120', 'SD4125', 'SD4297', 'SD5801',
    'SD5802', 'SD5805', 'SD5806', 'SD5807', 'SD5810', 'SD5820', 'SD5821',
    'SD5811', 'SD5813', 'SD5812']
    base_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2"
    driver = setup_driver()
    all_grades = {}
    all_summaries = []

    try:
        login_url = f"{base_url}/{module_codes[0]}/Final+grade/"
        manual_login(driver, login_url)

        for module_code in module_codes:
            print(f"\n🔍 Processing module {module_code}...")
            module_url = f"{base_url}/{module_code}/Final+grade/"
            driver.get(module_url)

            try:
                table_html = extract_table_html(driver)
                df = parse_html_table_to_dataframe(table_html)
                student_data, summary_row = filter_grades_dataframe(df, module_code)

                if student_data is not None:
                    all_grades[module_code] = student_data
                    all_summaries.append(summary_row)
                    print(f"✅ Data collected for {module_code}.")

            except Exception as e:
                print(f"⚠️ Error processing {module_code}: {e}")

        # Save everything to a single Excel file
        if all_grades:
            output_filename = "All_Modules_Grades_with_Summary.xlsx"
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                for module_code, df in all_grades.items():
                    df.to_excel(writer, sheet_name=module_code, index=False)

                if all_summaries:
                    summary_df = pd.concat(all_summaries)
                    summary_df.to_excel(writer, sheet_name="Summary")
            print(f"\n📁 All grades and summary saved to '{output_filename}'.")

    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()


if __name__ == "__main__":
    main()

And finally we can web scrapp the charts to compre current year grades vs previous years.

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service

from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
from PIL import Image as PILImage


def setup_driver():
    """Setup Edge driver"""
    edge_options = Options()
    # Keep browser visible for manual login
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=edge_options)
    return driver

def manual_authentication(driver):
    """Let user authenticate manually and confirm when ready"""
    print("=== MANUAL AUTHENTICATION ===")
    print("1. A browser window will open")
    print("2. Please log in to St Andrews manually")
    print("3. Navigate to any module page to confirm you're logged in")
    print("4. Come back here and press Enter when authentication is complete")
    print()
    
    # Open the login page
    test_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/GG1002/Final+grade/SubmitResults"
    print(f"Opening: {test_url}")
    driver.get(test_url)
    
    # Wait for user to complete authentication
    input("Press Enter after you have successfully logged in and can see the module page...")
    
    # Verify we're on the right page
    try:
        # Check if we can find elements that indicate we're logged in
        wait = WebDriverWait(driver, 5)
        
        # Look for page elements that confirm we're authenticated
        if "login" in driver.current_url.lower() or "auth" in driver.current_url.lower():
            print("Warning: Still appears to be on login page")
            input("Please complete login and press Enter again...")
        
        print("Authentication verified! Starting downloads...")
        return True
        
    except Exception as e:
        print(f"Note: {e}")
        print("Proceeding anyway...")
        return True

def save_charts_as_png(driver, url, module_code):
    """Navigate to URL and save scatterChart and barChart as PNG"""
    try:
        print(f"Loading {module_code}: {url}")
        driver.get(url)
        time.sleep(3)  # Wait for page to load
        
        # Check if we got redirected to login (shouldn't happen if session is valid)
        if "login" in driver.current_url.lower() or "auth" in driver.current_url.lower():
            print(f"Session expired! Please re-authenticate.")
            input("Complete authentication and press Enter...")
            driver.get(url)
            time.sleep(5)
        
        # Look specifically for scatterChart and barChart
        try:
            wait = WebDriverWait(driver, 15)
            
            # Find scatter chart
            scatter_chart = None
            bar_chart = None
            
            
            try:
                # scatter_chart = wait.until(EC.presence_of_element_located((By.ID, "scatterChart")))
                scatter_chart = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#scatterChart .user-select-none.svg-container")))
                print(f"  Found scatterChart for {module_code}")
            except:
                print(f"  scatterChart not found for {module_code}")
            """
            try:
                bar_chart = driver.find_element(By.ID, "barChart")
                print(f"  Found barChart for {module_code}")
            except:
                print(f"  barChart not found for {module_code}")
            """
            # Give charts extra time to fully render
            time.sleep(5)
            
            charts_found = []
            if scatter_chart:
                charts_found.append(("scatterChart", scatter_chart))
            if bar_chart:
                charts_found.append(("barChart", bar_chart))
            
            if not charts_found:
                print(f"No charts found for {module_code}")
                return 0
            
            print(f"Found {len(charts_found)} charts for {module_code}")
            
        except Exception as e:
            print(f"Error finding charts for {module_code}: {e}")
            return 0
        
        # Create folder for this module
        folder_path = os.path.join("charts", module_code)
        os.makedirs(folder_path, exist_ok=True)
        
        saved_count = 0
        
        # Save each chart with specific naming
        for i, (chart_type, chart_element) in enumerate(charts_found):
            try:
                # Scroll to chart to ensure it's visible
                driver.execute_script("arguments[0].scrollIntoView(true);", chart_element)
                time.sleep(2)
                
                # Take screenshot of the chart
                filename = os.path.join(folder_path, f"Chart_{i+1}.png")
                chart_element.screenshot(filename)
                
                print(f"  ✓ Saved Chart_{i+1}.png ({chart_type})")
                saved_count += 1
                
            except Exception as e:
                print(f"  ✗ Failed to save {chart_type}: {e}")
        
        print(f"Saved {saved_count} charts for {module_code}")
        return saved_count
        
    except Exception as e:
        print(f"Error processing {module_code}: {e}")
        return 0

def download_all_charts():
    """Main function - manual auth then download all charts"""
    
    print("St Andrews Module Charts Downloader")
    print("=" * 50)
    
    # Module codes to process
   
    module_codes = ['GG4258', 'GG3281', 'GG1002', 'GG2014', 'GG4248', 'GG4247', 'SS5103',
    'GG4254', 'GG4257', 'GG3205', 'GG3213', 'GG3214', 'GG5005', 'GG4399',
    'SD4126', 'SD4129', 'SD4133', 'SD1004', 'SD4225', 'SD2006', 'SD2100',
    'SD4110', 'SD3102', 'SD3101', 'SD4120', 'SD4125', 'SD4297', 'SD5801',
    'SD5802', 'SD5805', 'SD5806', 'SD5807', 'SD5810', 'SD5820', 'SD5821',
    'SD5811', 'SD5813', 'SD5812']
    
    #base_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/{}/Final+grade/GraphPage"
    base_url ="https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/{}/Final+grade/SubmitResults"
    
    # Create main charts folder
    os.makedirs("charts", exist_ok=True)
    
    # Setup browser
    print("Setting up browser...")
    driver = setup_driver()
    
    try:
        # Step 1: Manual authentication
        if not manual_authentication(driver):
            print("Authentication failed. Exiting...")
            return
        
        # Step 2: Download charts from all modules
        print(f"\nProcessing {len(module_codes)} modules...")
        print("=" * 30)
        
        total_saved = 0
        successful_modules = 0
        
        for i, module_code in enumerate(module_codes, 1):
            print(f"\n[{i}/{len(module_codes)}] Processing {module_code}...")
            
            url = base_url.format(module_code)
            saved = save_charts_as_png(driver, url, module_code)
            
            if saved > 0:
                successful_modules += 1
                total_saved += saved
            
            # Small delay between modules
            time.sleep(2)
        
        # Final summary
        print("\n" + "=" * 50)
        print("DOWNLOAD COMPLETE!")
        print(f"Processed: {len(module_codes)} modules")
        print(f"Successful: {successful_modules} modules")
        print(f"Total charts saved: {total_saved}")
        print(f"Charts saved in: ./charts/")
        print("=" * 50)
        
        
        # List what was downloaded
        print("\nDownloaded charts by module:")
        for module_code in module_codes:
            folder_path = os.path.join("charts", module_code)
            if os.path.exists(folder_path):
                files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
                if files:
                    print(f"  {module_code}: {len(files)} charts")
        
        generate_excel_from_charts("charts", "ModuleCharts.xlsx")
        
    except KeyboardInterrupt:
        print("\nDownload interrupted by user")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()
        

def generate_excel_from_charts(charts_dir="charts", output_file="ModuleCharts.xlsx"):
    """Creates an Excel file with one sheet per module, embedding saved PNG charts."""
    print("\nGenerating Excel file with charts...")
    wb = Workbook()
    wb.remove(wb.active)  # remove default sheet

    for module_code in os.listdir(charts_dir):
        module_path = os.path.join(charts_dir, module_code)
        if not os.path.isdir(module_path):
            continue

        sheet = wb.create_sheet(title=module_code)
        row_pos = 1

        # Sort to maintain order (e.g., Chart_1.png, Chart_2.png)
        for chart_file in sorted(os.listdir(module_path)):
            if not chart_file.lower().endswith('.png'):
                continue

            chart_path = os.path.join(module_path, chart_file)

            try:
                # Resize to avoid huge scaling in Excel
                img = PILImage.open(chart_path)
                img.thumbnail((1200, 800))  # Resize to max dimensions
                temp_path = chart_path.replace(".png", "_resized.png")
                img.save(temp_path)

                xl_img = XLImage(temp_path)
                cell_location = f"A{row_pos}"
                sheet.add_image(xl_img, cell_location)
                row_pos += 20  # space between charts

                print(f"  Added {chart_file} to sheet {module_code}")
            except Exception as e:
                print(f"  ✗ Failed to add {chart_file} to sheet {module_code}: {e}")

    wb.save(output_file)
    print(f"\n✓ Excel file created: {output_file}")

if __name__ == "__main__":
    download_all_charts()


To finish this a consolated version of the previous scripts, so we could create one excel file for the summary estadistics, charts and grades. with this excel, we can add the formula to define if the module report needs a comment or not, when it flagged. The script is also includedin in the repo as stand-alone script, so no need to run it as notebook. 

In [ ]:
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager

from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image as XLImage
from PIL import Image as PILImage


def setup_driver():
    """Setup Edge driver with appropriate options"""
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--window-size=1920,1080")
    service = Service(EdgeChromiumDriverManager().install())
    return webdriver.Edge(service=service, options=edge_options)


def manual_login(driver, test_url):
    """Handle manual authentication process"""
    print("=== Manual Authentication ===")
    print("1. A browser window will open")
    print("2. Please log in to St Andrews manually")
    print("3. Navigate to the grades page to confirm you're logged in")
    print("4. Come back here and press Enter when authentication is complete")
    print()
    
    driver.get(test_url)
    input("➡️ Once you're logged in and see the grades table, press Enter here...")
    
    # Verify authentication
    try:
        if "login" in driver.current_url.lower() or "auth" in driver.current_url.lower():
            print("Warning: Still appears to be on login page")
            input("Please complete login and press Enter again...")
        print("Authentication verified! Starting data extraction...")
        return True
    except Exception as e:
        print(f"Note: {e}")
        print("Proceeding anyway...")
        return True


def extract_table_html(driver):
    """Extract the grades table HTML"""
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gradesTable"))
    )
    table_element = driver.find_element(By.ID, "gradesTable")
    return table_element.get_attribute("outerHTML")


def parse_html_table_to_dataframe(table_html):
    """Parse HTML table into pandas DataFrame"""
    soup = BeautifulSoup(table_html, "html.parser")
    table = soup.find("table")
    df = pd.read_html(str(table), header=[0, 1])[0]  # Read MultiIndex headers
    return df


def filter_grades_dataframe(df, module_code):
    """Filter and process grades DataFrame"""
    print(f"📋 Processing columns for {module_code}...")

    try:
        filtered_df = df[[('Student ↓↑', 'Matric Number ↓↑'), ('Result ↓↑', 'Calc Grade ↓↑')]]
    except KeyError as e:
        print(f"❌ Columns not found in {module_code}. Error: {e}")
        return None, None

    filtered_df.columns = ['Matric Number', 'Calc Grade']

    # Split data from summary rows
    student_data = filtered_df.iloc[:-6].copy()
    summary_data = filtered_df.iloc[-6:].copy()

    # Convert grades to numeric (ignore non-numeric or missing values)
    student_data['Calc Grade'] = pd.to_numeric(student_data['Calc Grade'], errors='coerce')

    # Calculate extra statistics
    total_students = student_data['Calc Grade'].count()
    pct_gte_16_5 = (student_data['Calc Grade'] >= 16.5).sum() / total_students * 100 if total_students > 0 else 0
    pct_14_16 = ((student_data['Calc Grade'] >= 14) & (student_data['Calc Grade'] < 16)).sum() / total_students * 100 if total_students > 0 else 0

    # Format summary row from table
    summary_row = summary_data.set_index('Matric Number').T
    summary_row.columns.name = None
    summary_row['Module'] = module_code

    # Add new calculated percentages
    summary_row['% ≥ 16.5'] = round(pct_gte_16_5, 2)
    summary_row['% between 14–16'] = round(pct_14_16, 2)

    summary_row = summary_row.set_index('Module')

    return student_data, summary_row


def save_charts_as_png(driver, module_code, charts_dir="charts"):
    """Save both scatter charts for a module"""
    saved_count = 0
    
    # URLs for the two different scatter charts
    urls = {
        "GraphPage": f"https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/{module_code}/Final+grade/GraphPage",
        "SubmitResults": f"https://mms.st-andrews.ac.uk/mms/module/2024_5/S2/{module_code}/Final+grade/SubmitResults"
    }
    
    # Create folder for this module
    folder_path = os.path.join(charts_dir, module_code)
    os.makedirs(folder_path, exist_ok=True)
    
    for chart_type, url in urls.items():
        try:
            print(f"  Loading {chart_type} chart for {module_code}...")
            driver.get(url)
            time.sleep(3)  # Wait for page to load
            
            # Check if we got redirected to login
            if "login" in driver.current_url.lower() or "auth" in driver.current_url.lower():
                print(f"  Session expired! Please re-authenticate.")
                input("Complete authentication and press Enter...")
                driver.get(url)
                time.sleep(5)
            
            # Look for scatter chart
            try:
                wait = WebDriverWait(driver, 15)
                
                if chart_type == "GraphPage":
                    # Scatter chart 1 from GraphPage
                    scatter_chart = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#scatterChart .user-select-none.svg-container")))
                    chart_name = "ScatterChart_1"
                else:
                    # Scatter chart 2 from SubmitResults
                    scatter_chart = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#scatterChart .user-select-none.svg-container")))
                    chart_name = "ScatterChart_2"
                
                print(f"    Found {chart_name} for {module_code}")
                
                # Give chart extra time to fully render
                time.sleep(5)
                
                # Scroll to chart to ensure it's visible
                driver.execute_script("arguments[0].scrollIntoView(true);", scatter_chart)
                time.sleep(2)
                
                # Take screenshot of the chart
                filename = os.path.join(folder_path, f"{chart_name}.png")
                scatter_chart.screenshot(filename)
                
                print(f"    ✓ Saved {chart_name}.png")
                saved_count += 1
                
            except Exception as e:
                print(f"    ✗ {chart_name} not found for {module_code}: {e}")
        
        except Exception as e:
            print(f"  Error processing {chart_type} for {module_code}: {e}")
    
    return saved_count


def add_charts_to_excel(wb, module_code, charts_dir="charts"):
    """Add charts to the existing module sheet in the workbook"""
    try:
        # Get the existing sheet for this module
        if module_code in wb.sheetnames:
            sheet = wb[module_code]
        else:
            return False
        
        # Find the last row with data to position charts below
        last_row = sheet.min_row   # Add some spacing
        
        # Look for chart files for this module
        module_path = os.path.join(charts_dir, module_code)
        if not os.path.exists(module_path):
            return False
        
        chart_files = [f for f in os.listdir(module_path) if f.endswith('.png') and not f.endswith('_resized.png')]
        
        if not chart_files:
            return False
        
        # Sort chart files for consistent ordering
        chart_files.sort()
        
        current_row = last_row
        for chart_file in chart_files:
            chart_path = os.path.join(module_path, chart_file)
            
            try:
                # Check if original file exists
                if not os.path.exists(chart_path):
                    print(f"    ✗ Chart file not found: {chart_path}")
                    continue
                
                # Resize image to reasonable size for Excel
                img = PILImage.open(chart_path)
                img.thumbnail((800, 600))  # Resize to max dimensions
                temp_path = chart_path.replace(".png", "_resized.png")
                img.save(temp_path)
                
                # Verify temp file was created
                if not os.path.exists(temp_path):
                    print(f"    ✗ Failed to create resized image: {temp_path}")
                    continue
                
                # Add to Excel sheet
                xl_img = XLImage(temp_path)
                cell_location = f"D{current_row}"
                sheet.add_image(xl_img, cell_location)
                current_row += 25  # Space between charts
                
                print(f"    ✓ Added {chart_file} to {module_code} sheet")
   
            except Exception as e:
                print(f"    ✗ Failed to add {chart_file} to {module_code} sheet: {e}")
        
        return True
        
    except Exception as e:
        print(f"  Error adding charts to {module_code} sheet: {e}")
        return False


def main():
    """Main function"""
    print("St Andrews Module Data and Charts Extractor")
    print("=" * 60)
    
    # Module codes to process
    module_codes = ['GG4258', 'GG3281', 'GG1002', 'GG2014', 'GG4248', 'GG4247', 'SS5103',
                    'GG4254', 'GG4257', 'GG3205', 'GG3213', 'GG3214', 'GG5005', 'GG4399',
                    'SD4126', 'SD4129', 'SD4133', 'SD1004', 'SD4225', 'SD2006', 'SD2100',
                    'SD4110', 'SD3102', 'SD3101', 'SD4120', 'SD4125', 'SD4297', 'SD5801',
                    'SD5802', 'SD5805', 'SD5806', 'SD5807', 'SD5810', 'SD5820', 'SD5821',
                    'SD5811', 'SD5813', 'SD5812']
    
    base_url = "https://mms.st-andrews.ac.uk/mms/module/2024_5/S2"
    output_filename = "Complete_Modules_Data_and_Charts.xlsx"
    charts_dir = "charts"
    
    # Create charts directory
    os.makedirs(charts_dir, exist_ok=True)
    
    # Setup driver
    driver = setup_driver()
    all_grades = {}
    all_summaries = []
    
    try:
        # Step 1: Manual login
        login_url = f"{base_url}/{module_codes[0]}/Final+grade/"
        manual_login(driver, login_url)
        
        print(f"\n🔍 Processing {len(module_codes)} modules...")
        print("=" * 40)
        
        # Step 2: Extract grades data and charts for each module
        total_charts_saved = 0
        successful_modules = 0
        
        for i, module_code in enumerate(module_codes, 1):
            print(f"\n[{i}/{len(module_codes)}] Processing module {module_code}...")
            
            # Extract grades data
            try:
                module_url = f"{base_url}/{module_code}/Final+grade/"
                driver.get(module_url)
                
                table_html = extract_table_html(driver)
                df = parse_html_table_to_dataframe(table_html)
                student_data, summary_row = filter_grades_dataframe(df, module_code)
                
                if student_data is not None:
                    all_grades[module_code] = student_data
                    all_summaries.append(summary_row)
                    print(f"  ✅ Grades data collected for {module_code}")
                else:
                    print(f"  ⚠️ No grades data for {module_code}")
                    
            except Exception as e:
                print(f"  ⚠️ Error extracting grades for {module_code}: {e}")
            
            # Extract charts
            try:
                charts_saved = save_charts_as_png(driver, module_code, charts_dir)
                if charts_saved > 0:
                    total_charts_saved += charts_saved
                    successful_modules += 1
                    print(f"  ✅ {charts_saved} charts saved for {module_code}")
                else:
                    print(f"  ⚠️ No charts saved for {module_code}")
            except Exception as e:
                print(f"  ⚠️ Error extracting charts for {module_code}: {e}")
            
            # Small delay between modules
            time.sleep(2)
        
        # Step 3: Create Excel workbook with grades data
        if all_grades:
            print(f"\n📊 Creating Excel workbook with grades and charts...")
            
            with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
                # Write individual module sheets with grades
                for module_code, df in all_grades.items():
                    df.to_excel(writer, sheet_name=module_code, index=False)
                
                # Write summary sheet
                if all_summaries:
                    summary_df = pd.concat(all_summaries)
                    summary_df.to_excel(writer, sheet_name="Summary")
            
            # Step 4: Add charts to the existing workbook
            print("📈 Adding charts to Excel sheets...")
            wb = load_workbook(output_filename)
            
            charts_added = 0
            for module_code in all_grades.keys():
                if add_charts_to_excel(wb, module_code, charts_dir):
                    charts_added += 1
            
            wb.save(output_filename)
            
            # Final summary
            print("\n" + "=" * 60)
            print("EXTRACTION COMPLETE!")
            print(f"Processed modules: {len(module_codes)}")
            print(f"Modules with grades: {len(all_grades)}")
            print(f"Modules with charts: {successful_modules}")
            print(f"Total charts saved: {total_charts_saved}")
            print(f"Sheets with charts added: {charts_added}")
            print(f"Output file: {output_filename}")
            print("=" * 60)
            
        else:
            print("❌ No grades data collected. Please check authentication and module URLs.")
    
    except KeyboardInterrupt:
        print("\nProcess interrupted by user")
    except Exception as e:
        print(f"Unexpected error: {e}")
        #import traceback
        #print("Full error traceback:")
        #traceback.print_exc()
    finally:
        input("\nPress Enter to close the browser...")
        driver.quit()
        
        # Clean up any remaining temporary chart files
        print("Cleaning up temporary files...")
        try:
            for module_code in module_codes:
                module_path = os.path.join(charts_dir, module_code)
                if os.path.exists(module_path):
                    for file in os.listdir(module_path):
                        if file.endswith('_resized.png'):
                            temp_file_path = os.path.join(module_path, file)
                            try:
                                if os.path.exists(temp_file_path):
                                    os.remove(temp_file_path)
                                    print(f"  Removed: {temp_file_path}")
                            except Exception as cleanup_error:
                                print(f"  Could not remove {temp_file_path}: {cleanup_error}")
        except Exception as e:
            print(f"Note: Error during cleanup: {e}")

if __name__ == "__main__":
    main()